<div style="background: GRAY">
<div style="text-align: center">
<br>
<h1>ETL Preparation</h1>
<h4>Import packages, load data, define helper functions</h4>
<br>
</div></div>

## Imports and loading

In [1]:
# custom styling
from IPython.core.display import HTML
HTML('''<style>.container { width:100%; }</style>''')

In [3]:
from pymongo import MongoClient
client = MongoClient()
client = MongoClient("mongodb://localhost:27017")
db = client.reporter

In [7]:
def loadDF(df, collectionName, dropFirst=True):
    if dropFirst:
        db[collectionName].drop()
    db[collectionName].insert_many(df.to_dict(orient='records'))
    print 'Successfully loaded:', collectionName

In [6]:
import pandas as pd

In [688]:
df12raw = pd.read_excel('RePORTER_PRJ_C_FY2012.xlsx')
df12 = df12raw.copy()

In [687]:
df15raw = pd.read_excel('RePORTER_PRJ_C_FY2015.xlsx')
df15 = df15raw.copy()

<div style="background: linen">
<div style="text-align: center">
<br>
<h1>DATA CLEANING</h1>
<h4>Transformations to reformat &, || reorganize</h4>
<br>
</div></div>

### Funding IC Names and Dollars Contrib'd

In [918]:
# parse the IC names and costs into an array
def fundingSplitter(icArray):
    icArray = str(icArray).split('''\\''')
    extractedICs = []
    for ic in icArray:
        if ic:
            tmp = ic.split(''':''')
            icName = tmp[0] if tmp[0] else ''
            cost = tmp[1] if len(tmp)>1 else 0
            extractedICs.append({'IC': icName, 'Cost': cost})
    return extractedICs
df12['FUNDING_PARSED'] = df12.FUNDING_ICs.apply(fundingSplitter)
df15['FUNDING_PARSED'] = df15.FUNDING_ICs.apply(fundingSplitter)
df12 = df12.drop('FUNDING_ICs', axis=1)
df15 = df15.drop('FUNDING_ICs', axis=1)

### Projects with Sub Project records have a sub-total row with Total Cost
*These sub-total rows need to be removed*

In [919]:
# need to know whether there are subprojects to remove the total_cost row which will cause cost duplication
nSubProjects = df15[['FULL_PROJECT_NUM','TOTAL_COST_SUB_PROJECT']]\
    .groupby('FULL_PROJECT_NUM')\
    .apply(lambda proj: proj['TOTAL_COST_SUB_PROJECT'].nunique())\
    .sort_values(ascending=False)\
    .reset_index()\
    .rename(columns={0: 'subProjects'})
# merge in nSubs
df15 = df15.merge(nSubProjects, how='left')
# filter out total cost rows of projects with existing subprojects
df15 = df15[((df15.subProjects>0) & (df15.TOTAL_COST.isnull())) | (df15.subProjects==0)].drop('subProjects', axis=1)

### Spending Categories and Project Terms

In [920]:
def termSplitter(arr):
    terms = str(arr).split(';') if not pd.isnull(arr) else []
    return [term.lower().strip() for term in terms]
df12['NIH_SPENDING_CATS_PARSED'] = df12.NIH_SPENDING_CATS.map(termSplitter)
df12['PROJECT_TERMS_PARSED'] = df12.PROJECT_TERMS.map(termSplitter)
df15['NIH_SPENDING_CATS_PARSED'] = df12.NIH_SPENDING_CATS.map(termSplitter)
df15['PROJECT_TERMS_PARSED'] = df12.PROJECT_TERMS.map(termSplitter)
df12 = df12.drop(['NIH_SPENDING_CATS','PROJECT_TERMS'], axis=1)
df15 = df15.drop(['NIH_SPENDING_CATS','PROJECT_TERMS'], axis=1)

### Researchers and PI IDs

In [921]:
# can use termSplitter() as well
df12['PI_NAMES_PARSED'] = df12.PI_NAMEs.map(lambda name: name.encode('utf-8')).map(termSplitter)
df12['PI_IDS_PARSED'] = df12.PI_IDS.map(lambda name: name.encode('utf-8')).map(termSplitter)
df15['PI_NAMES_PARSED'] = df15.PI_NAMEs.map(lambda name: name.encode('utf-8')).map(termSplitter)
df15['PI_IDS_PARSED'] = df15.PI_IDS.map(lambda name: name.encode('utf-8')).map(termSplitter)
df12 = df12.drop('PI_NAMEs', axis=1)
df15 = df15.drop('PI_NAMEs', axis=1)

<div style="background: steelblue">
<div style="text-align: center">
<br>
<h1>MODELING</h1>
<h4>Define, extract model, load into mongo</h4>
<br>
</div></div>

### For FY15 only right now

In [922]:
df = df15

<div style="background: lightgreen">
<h3>Model: Spending Categories, Project Terms (dfSpendingCats, dfProjectTerms)</h3>
</div>
Project Terms is 1.6M records large... leaving out for now

In [923]:
def getUniqueFromSeriesOfArrays(df, column):
    items = []
    series = df[df[column].map(lambda r: True if len(r) else False)][column] # filter empty arrays
    series.map(lambda arr: map(lambda i: items.append(i), arr)) # fill items array
    uniques = [i for i in list(set(items)) if i] # return only real values
    uniques.sort()
    return uniques

In [924]:
# get unique spending categories, and project terms
dfSpendingCats = pd.DataFrame(getUniqueFromSeriesOfArrays(df, 'NIH_SPENDING_CATS_PARSED'), columns=['TERM']).reset_index().rename(columns={"index":"_id"})
#dfProjectTerms = pd.DataFrame(getUniqueFromSeriesOfArrays(df, 'PROJECT_TERMS_PARSED'), columns=['term']).reset_index().rename(columns={"index":"_id"})

In [925]:
# map grants to spending cats/proj terms
dfSpendingCats['GRANTS'] = dfSpendingCats.TERM.map(lambda term: df['APPLICATION_ID'][df.NIH_SPENDING_CATS_PARSED.map(lambda l: term in l)].tolist())
#dfProjectTerms['GRANTS'] = dfProjectTerms.term.map(lambda term: df['APPLICATION_ID'][df.PROJECT_TERMS_PARSED.map(lambda l: term in l)].tolist())

In [927]:
# map dfSpendingCats/dfProjectTerms._id --> df [long processing time (hrs)]
def mapDFArrayToRef(dfArray, refDf):
    if dfArray:
        return [int(refDf.query('TERM == @cat')['_id'].values[0]) for cat in dfArray if cat]
    else:
        return []
df['SPENDING_CATEGORIES'] = df.NIH_SPENDING_CATS_PARSED.map(lambda spendingArray: mapDFArrayToRef(spendingArray, dfSpendingCats))
#df['PROJECT_TERMS'] = df.PROJECT_TERMS_PARSED.map(lambda spendingArray: mapDFArrayToRef(spendingArray, dfProjectTerms))
df = df.drop('NIH_SPENDING_CATS_PARSED', axis=1)
#df = df.drop('PROJECT_TERMS_PARSED', axis=1)

In [928]:
# prep for load
dfSpendingCats = dfSpendingCats.rename(columns={'TERM':'term','GRANTS':'grants'})
# load Spending/Project terms into Mongo
loadDF(dfSpendingCats, 'spendingcategory')
#loadDF(dfProjectTerms, 'projectterms')

Successfully loaded: spendingcategory


<div style="background: lightgreen">
<h3>Model: Researchers (dfPIs)</h3>
</div>

In [930]:
PIs = []
for r in df[['PI_NAMES_PARSED','PI_IDS_PARSED']].iterrows():
    namesArr = r[1][0] # get list of names
    idArr = r[1][1] # get list of IDs
    namesArr = [i for i in namesArr if i] # filter empties
    idArr = [i for i in idArr if i] # filter empties
    d = dict(zip(idArr, namesArr)) # make dict
    for k, v in d.iteritems(): # iter through dict and drop 'contact' designator
        PIs.append({'_id': k.rstrip(' (contact)'),'NAME':v.rstrip(' (contact)').replace('.','')})
dfPIs = pd.DataFrame(PIs).drop_duplicates()
dfPIs['_id'] = dfPIs._id.apply(int)
dfPIs['NAME'] = dfPIs.NAME.apply(str)

In [931]:
# clean up IDS by removing " (contact)"
df['PI_IDS_PARSED'] = df.PI_IDS_PARSED.apply(lambda x: [int(i.rstrip(' (contact)')) for i in x if i])

In [932]:
# mapping application_ids to dfPIs
dfPIs['GRANTS'] = dfPIs._id.apply(int).map(lambda y: df[['APPLICATION_ID','PI_IDS_PARSED']][df.PI_IDS_PARSED.map(lambda x: y in x)].APPLICATION_ID.tolist())

In [933]:
# pi_ids already mapped due to extraction in cleaning phase, just need to clean/drop precursor cols
df = df.rename(columns={'PI_IDS_PARSED':'PIs'}).drop(['PI_IDS','PI_NAMES_PARSED'],axis=1)

In [935]:
dfPIs = dfPIs.rename(columns={'NAME':'name','GRANTS':'grants'})
# load researchers
loadDF(dfPIs, 'researcher')

Successfully loaded: researcher


<div style="background: lightgreen">
<h3>Model: Organizations (dfOrgs)</h3>
</div>
Attributes: __ED_INST_TYPE, ORG_CITY, ORG_COUNTRY, ORG_DEPT, ORG_DISTRICT, ORG_DUNS, ORG_FIPS, ORG_NAME, ORG_STATE, ORG_ZIPCODE__

In [936]:
orgCols = ['ED_INST_TYPE','ORG_CITY','ORG_COUNTRY','ORG_DEPT','ORG_DISTRICT','ORG_DUNS','ORG_FIPS','ORG_NAME','ORG_STATE','ORG_ZIPCODE']
orgs = df[orgCols].drop_duplicates()

# unique ID used is ORG_NAME, so everything else should be collated under that
def pivotField(arr):
    tmp = arr.unique().tolist()
    if len(tmp)==1:
        return tmp[0]
    else:
        return tmp

dfOrgs = orgs.groupby('ORG_NAME').agg(pivotField).reset_index()\
    .rename(columns={
        'ORG_DUNS':'DUNS',
        'ED_INST_TYPE':'TYPE',
        'ORG_CITY':'CITY',
        'ORG_COUNTRY':'COUNTRY',
        'ORG_DEPT':'DEPARTMENT',
        'ORG_DISTRICT':'DISTRICT',
        'ORG_FIPS':'FIPS',
        'ORG_NAME':'NAME',
        'ORG_STATE':'STATE',
        'ORG_ZIPCODE':'ZIP'
    })
dfOrgs = dfOrgs.reset_index().rename(columns={"index":"_id"})

In [937]:
# map APP_IDS to dfOrgs
dfOrgs['GRANTS'] = dfOrgs.NAME.apply(lambda i: df[['APPLICATION_ID', 'ORG_NAME']][df[['APPLICATION_ID','ORG_NAME']].ORG_NAME == i].APPLICATION_ID.unique().tolist())

In [938]:
# map dfOrgs_id to df
df = df.merge(dfOrgs[['_id','NAME']], how='left', left_on='ORG_NAME', right_on='NAME').drop('NAME', axis=1).drop(orgCols, axis=1).rename(columns={'_id':'ORGANIZATION'})

In [941]:
dfOrgs = dfOrgs.rename(columns={'NAME':'name','TYPE':'type','CITY':'city','COUNTRY':'county','DEPARTMENT':'department','DISTRICT':'district','DUNS':'duns','FIPS':'fips',\
                               'STATE':'state','ZIP':'zip','GRANTS':'grants'})
# load organizations
loadDF(dfOrgs, 'organization')

Successfully loaded: organization


<div style="background: lightgreen">
<h3>Model: Funders (dfICs)</h3>
</div>
Attributes: __IC_NAME, ADMINISTERING_IC__

*See [NIH Funders Dictionary](https://www.nlm.nih.gov/bsd/grant_acronym.html) for reference*

In [965]:
dfICs = pd.read_excel('ICs.xlsx').reset_index().rename(columns={'index':'_id','Agency':'AGENCY','Grant Abbreviation':'ABBREVIATION','Acronym':'ACRONYM','Full Institute/Organization Name':'IC_NAME'})

In [967]:
# check to make sure all dfIC records are in current DF
tmp = df.FUNDING_PARSED.map(lambda r: [i['IC'] for i in r]).tolist() # collect all ICs involved in funding rounds
FUNDING_ABBRVS = list(set([abbrv for i in tmp for abbrv in i])) # collapse list of lists and select uniques
sum([(i not in dfICs.ABBREVIATION.tolist())*1 for i in df.ADMINISTERING_IC.unique().tolist()]) # check to make sure all abbreviations are in the list (should be 0)

In [944]:
# map dfICs ids into df
df = df.merge(dfICs[['_id','ABBREVIATION']], how='left', left_on='ADMINISTERING_IC', right_on='ABBREVIATION')\
    .drop(['ABBREVIATION','ADMINISTERING_IC','IC_NAME'], axis=1)\
    .rename(columns={'_id':'ADMINISTERING_IC'})

In [971]:
# map df APP_IDs to dfICs
dfICs['ADMINISTERING_OVER'] = dfICs._id.apply(lambda ic: df[['APPLICATION_ID','ADMINISTERING_IC']][df[['APPLICATION_ID','ADMINISTERING_IC']].ADMINISTERING_IC == ic].APPLICATION_ID.unique().tolist())

In [972]:
# loading happens after funding since dfICs needs to be mapped to dfFunding records
dfICs = dfICs.rename(columns={'AGENCY':'agency','ABBREVIATION':'abbreviation','ACRONYM':'acronym','IC_NAME':'icName','GRANTS':'grantsAdministering'})

<div style="background: lightgreen">
<h3>Model: Funding</h3>
</div>

In [976]:
fundings = df.reset_index().rename(columns={'index':'FUNDING_ID'})[['FUNDING_ID','APPLICATION_ID','FUNDING_PARSED']]

In [952]:
ids = []
appIds = []
fIC = []
amt = []
for r in fundings.iterrows():
    for i in r[1].FUNDING_PARSED:
        ids.append(r[1].FUNDING_ID)
        appIds.append(r[1].APPLICATION_ID)
        fIC.append(i['IC'])
        amt.append(i['Cost'])
dfFunding = pd.DataFrame(zip(ids, appIds, fIC, amt), columns=['FUNDING_ROUND','GRANTS','FUNDING_IC','FUNDING_AMOUNT']).reset_index().rename(columns={'index':'_id'})

In [953]:
# map funding IDs to df
df['FUNDINGS'] = dfFunding.groupby('FUNDING_ROUND')['_id'].apply(lambda x: x.tolist())
df = df.drop('FUNDING_PARSED',axis=1)
dfFunding = dfFunding.rename(columns={'FUNDING_ROUND':'fundingRound','GRANTS':'grant','FUNDING_IC':'fundingIc','FUNDING_AMOUNT':'fundingAmount'})

In [1004]:
# remap OD acronym
dfFunding = dfFunding.replace({'fundingIc': {'OD':'NIH-OD'}})

In [1016]:
# map dfIC ids to dfFunding
dfFunding = dfFunding\
    .merge(dfICs[['_id','acronym']], how='left', left_on='fundingIc', right_on='acronym')\
    .drop(['fundingIc','acronym'], axis=1)\
    .rename(columns={'_id_y':'fundingIc','_id_x':'_id'})

In [1025]:
groupedICIds = dfFunding[['_id','fundingIc']].groupby('fundingIc')['_id'].apply(lambda x: x.tolist())

In [1039]:
dfICs = dfICs\
    .merge(groupedICIds.reset_index(), how='left', left_on='_id', right_on='fundingIc')\
    .drop('fundingIc',axis=1)\
    .rename(columns={'_id_x':'_id','_id_y':'fundings'})

In [1041]:
loadDF(dfFunding, 'funding')
loadDF(dfICs, 'institute')

Successfully loaded: funding
Successfully loaded: institute


<div style="background: lightgreen">
<h3>Model: Projects</h3>
</div>

In [1042]:
dfProjects = df[['ACTIVITY','ADMINISTERING_IC','CFDA_CODE','FOA_NUMBER','SERIAL_NUMBER','SUFFIX','CORE_PROJECT_NUM','FULL_PROJECT_NUM']].drop_duplicates()
# map APP_IDs to dfProj
dfProjects = dfProjects.merge(
    df[['FULL_PROJECT_NUM','APPLICATION_ID']].groupby('FULL_PROJECT_NUM')['APPLICATION_ID'].apply(lambda x: x.tolist()).reset_index(),
    how='left')\
    .rename(columns={'APPLICATION_ID':'GRANTS','FULL_PROJECT_NUM':'_id'})

In [1044]:
# clean up cols
df = df.drop(['ACTIVITY','ADMINISTERING_IC','CFDA_CODE','FOA_NUMBER','SERIAL_NUMBER','SUFFIX','CORE_PROJECT_NUM'], axis=1)

In [1047]:
dfProjects = dfProjects.rename(columns={'ACTIVITY':'activity','ADMINISTERING_IC':'administeringIc','CFDA_CODE':'cfdaCode','FOA_NUMBER':'foaNumber',\
                                        'SERIAL_NUMBER':'serialNumber','SUFFIX':'suffix','CORE_PROJECT_NUM':'coreProjectNum','GRANTS':'grants'})
loadDF(dfProjects, 'project')

Successfully loaded: project


<div style="background: lightgreen">
<h3>Model: Grants</h3>
</div>

In [1051]:
cols = {
 'APPLICATION_ID': '_id',
 'APPLICATION_TYPE': 'applicationType',
 'ARRA_FUNDED': 'arraFunded',
 'AWARD_NOTICE_DATE': 'awardNoticeDate',
 'BUDGET_START':'budgetStart',
 'BUDGET_END': 'budgetEnd',
 'FUNDING_MECHANISM': 'fundingMechanism',
 'FY': 'fy',
 'PHR': 'phr',
 'PROGRAM_OFFICER_NAME': 'programOfficerName',
 'PROJECT_START': 'projectStart',
 'PROJECT_END': 'projectEnd',
 'PROJECT_TITLE': 'projectTitle',
 'STUDY_SECTION': 'studySection',
 'STUDY_SECTION_NAME': 'studySectionName',
 'SUBPROJECT_ID': 'subprojectId',
 'SUPPORT_YEAR': 'supportYear',
 'DIRECT_COST_AMT': 'directCostAmt',
 'INDIRECT_COST_AMT': 'indirectCostAmt',
 'TOTAL_COST': 'totalCost',
 'TOTAL_COST_SUB_PROJECT': 'totalCostSubProject',
 'PROJECT_TERMS_PARSED': 'projectTerms',
 'PIs': 'pis',
 'SPENDING_CATEGORIES': 'spendingCategories',
 'ORGANIZATION': 'organization',
 'FUNDINGS': 'fundings',
 'FULL_PROJECT_NUM': 'project'
}

In [1061]:
# mongo can't handle NaT null type
df[['AWARD_NOTICE_DATE', 'BUDGET_START', 'BUDGET_END', 'PROJECT_START', 'PROJECT_END']] = \
    df[['AWARD_NOTICE_DATE', 'BUDGET_START', 'BUDGET_END', 'PROJECT_START', 'PROJECT_END']].fillna(value=' ')
dfGrants = df.rename(columns=cols)

In [1065]:
loadDF(dfGrants,'grant')

Successfully loaded: grant
